# Machine Learning Pipeline with KubeDirector - Lab 2
## Deploy a local Jupyter Notebook cluster to interact with a tenant-shared training cluster

### **Lab workflow:**

In this lab:

1. As tenant user, you will first create a local (lightweight) Jupyter Notebook application KubeDirector cluster to develop your model. You will attach your Jupyter Notebook cluster to a remote tenant-shared training cluster to train your model. The shared training cluster **training-engine-shared** includes the open source ML toolkits, libraries and frameworks for developing and training models. It has been already deployed by the tenant administrator for your tenant. The shared training cluster will allow you to train your model faster using more compute and memory resources than your local lightweight Jupyter Notebook cluster.

2. You will then access your local Jupyter Notebook web UI via the gateway network service port to train your model to the remote tenant-shared training cluster. 

**Recommended blog reading:**

* [Building Dynamic Machine Learning Pipelines with KubeDirector](https://developer.hpe.com/blog/building-dynamic-machine-learning-pipelines-with-kubedirector)

**Definitions:**

- *KubeDirector:* also known as Kubernetes Director. KubeDirector is an **open-source** project initiated and led by HPE that addresses stateful scaleout application deployment in standard Kubernetes clusters with a focus on non-cloud native stateful analytics workloads (AI/ML, data processing and Big Data apps). These applications generally refer to as a distributed, single-node or multi-node application **virtual cluster** where each application virtual cluster node runs as **a container** in the Kubernetes cluster.

- *Training:* Input datasets are processed to create Machine Learning Model. Data Scientists can use local Jupyter Notebook to build their models and train their models. They can also interact with remote larger capacity training cluster to train their models faster.

- *Cloud native application:* Also known as the [12-Factor app](https://www.mirantis.com/blog/how-do-you-build-12-factor-apps-using-kubernetes/), a modern application that leverages microservices architecture with loosely coupled services. The microservice architectural style is an approach to developing a single application as a suite of small independently deployed services.

- *Non-cloud native application:* a multi-tier application with tightly coupled and interdependent services. 

- *Stateless application:* A stateless application is an application which does not require persistence of data nor an application state.

- *Stateful application:* A stateful application typically requires persistence of certain mountpoints across application cluster nodes rescheduling, restarts, upgrades, rollbacks. A stateful application can also need persistence of network identity (i.e.: hostname). 

#### **Download the python code files to your PC/laptop:**

From the left side panel, navigate to **Code/NYCTaxi** folder and download the files:
- _**3-WKSHP-K8s-ML-Pipeline-Model-Training.ipynb**_
- _**XGB_Scoring.py**_
- _**XGB_Scoringv2.py**_
- _**ML-Workflow.jpg**_

Double-click on the folder **Code**, then folder **NYCTaxi**. Right-click on each code file and choose **Download**.

You will need these files in the next lab.

You can click the ellipsis **(...)** to go back to the root of your repository in JupyterHub.

#### Initialize the environment:

Let's first define the environment variables needed to execute this lab part.

In [1]:
#
# environment variables to be verified by the student
#
studentId="student{{ STDID }}" # your Jupyter Notebook student Identifier (i.e.: student<xx>)

studentId="student74"

# fixed environment variables setup by the HPE ECP lab administrator - Please DO NOT MODIFY!!

gateway_host="{{ HPEECPGWNAME }}"
Internet_access="{{ JPHOSTEXT }}"

gateway_host="hpecpgw1.hp.local"
Internet_access="notebooks2.hpedev.io"

JupyterNotebookApp="cr-cluster-jupyter-notebook.yaml" # the Jupyter Notebook KD App manifest you will deploy to build your model
DeploymentEngineApp="cr-cluster-endpoint-wrapper.yaml" # the Deployment engine KD App manifest you will deploy to query your model for answers

echo "Your studentId is: "$studentId

Your studentId is: student74


#### List the registered KubeDirector Applications:
You can get the list of KubeDirector Applications (kdapp) registered with the Kubernetes cluster for your Tenant using the following kubectl command. A KubeDirector Application is a _template_ for the application. It describes an application's **metadata** (service roles, Docker images, configuration packages, services ports, persistent storage).
In this workshop, you will be using the following KubeDirector Applications: _jupyter-notebook-v1_ to build your model and _deployment-engine_ app to deploy your trained model and serve prediction queries.

In [2]:
kubectl get kdapp

NAME                     AGE
centos8x                 22h
deployment-engine        22h
jupyter-notebook         22h
jupyter-notebook-v1      22h
spark245                 22h
tensorflow-gpu-jupyter   22h
training-engine          22h
ubuntu18x                22h


#### List the existing KubeDirector Training cluster deployed on the Kubernetes cluster for your tenant:

In [3]:
kubectl get kdcluster training-engine-shared

NAME                     AGE
training-engine-shared   22h


## Deploying your local Jupyter Notebook cluster with Connection to the Training cluster

#### Deploy an instance of the Jupyter Notebook KubeDirector application:
You will deploy an instance of the **jupyter-notebook** KubeDirector application (kdapp) with the Training cluster _Connection_ by creating a KubeDirector virtual cluster (kdcluster). A kdcluster identifies the desired kdapp and specifies runtime configuration parameters, such as the size and resource requirements of the virtual cluster. 

> **Note:** _The Jupyter-Notebook kdapp includes the open source Machine learning toolkits, software libraries and frameworks for developing and training models such as TensorFlow, scikit-learn, keras, XGBoost, matplotlib, Jupyter Notebook, Numpy, Scipy, Pandas..._

Like any other containerized application deployment on Kubernetes, the K8s API call requires the kubectl operation type (create or apply) and the application manifest (a YAML file that describes the attributes of the application).  

As you are all sharing the same tenant context and Kubernetes cluster resources, let's make sure your application deployment name will be unique among the tenant users. Here we replace the string "example" with your "studentId" in the application manifest file.

In [4]:
sed -i "s/example/${studentId}/g" $JupyterNotebookApp
cat $JupyterNotebookApp

apiVersion: "kubedirector.hpe.com/v1beta1"
kind: "KubeDirectorCluster"
metadata:
  name: "jupyter-notebook-student74"
spec:
  app: "jupyter-notebook-v1"
  appCatalog: "local"
  connections: 
    #secrets: 
      #- 
        #"some secrets"
    #configmaps: 
      #- 
        #"some configmaps"
    clusters: 
      - "training-engine-shared"
        #"some clusters"
  roles:
  - id: controller
    resources:
      requests:
        memory: "2Gi"
        cpu: "1"
      limits:
        memory: "2Gi"
        cpu: "1"


> **Note:** _one of the most interesting parts of the kdcluster spec is the **Connections** stanza (a related group of attributes), which identifies other resources of interest to that kdcluster. Here, you simply connect your local Jupyter Notebook cluster to the tenant-shared training cluster **training-engine-shared** already deployed by the tenant administrator._

In [5]:
kubectl apply -f $JupyterNotebookApp

kubedirectorcluster.kubedirector.hpe.com/jupyter-notebook-student74 created


After a few seconds, you should get the response message to your K8s API request: *kubedirectorcluster/Your-instance-name created*.  

In the next steps, you will use kubectl commands in the context of your tenant user account and get the Notebook service endpoint with token-based authorization to connect to it.

#### Inspect the deployed KubeDirector application instance: 
Your application will be represented in the Kubernetes cluster by a custom resource of type **KubeDirectorCluster (kdcluster)**, with the name that was indicated inside the YAML file used to create it. 

In [6]:
clusterName="jupyter-notebook-${studentId}"
kubectl get kdcluster $clusterName

NAME                         AGE
jupyter-notebook-student74   5s


After creating the instance of the KubeDirector Application, you can use the `kubectl describe kdcluster` command below to observe its status and the standard Kubernetes resources that compose the application virtual cluster (statefulsets, pods, services, persistent volume claim if any), as well as any events logged against it.

The virtual cluster status indicates its overall "state" (top-level property of the status object). It should have a value of **"configured"**. 

> **Note:** _The first time a virtual cluster of a given KubeDirector Application type is created, it may take some minutes to reach its **"configured"** state, as the relevant Docker image must be downloaded and imported._ 

**Repeat the command below until the kdcluster is in state "configured"**

In [10]:
kubectl describe kdcluster $clusterName

Name:         jupyter-notebook-student74
Namespace:    k8smltenant
Labels:       <none>
Annotations:  <none>
API Version:  kubedirector.hpe.com/v1beta1
Kind:         KubeDirectorCluster
Metadata:
  Creation Timestamp:  2020-12-16T18:25:11Z
  Finalizers:
    kubedirector.hpe.com/cleanup
  Generation:        1
  Resource Version:  307974
  Self Link:         /apis/kubedirector.hpe.com/v1beta1/namespaces/k8smltenant/kubedirectorclusters/jupyter-notebook-student74
  UID:               7afed7a4-454d-48f4-8ada-2511dcc4f1ae
Spec:
  App:          jupyter-notebook-v1
  App Catalog:  local
  Connections:
    Clusters:
      training-engine-shared
  Naming Scheme:  UID
  Roles:
    Id:       controller
    Members:  1
    Resources:
      Limits:
        Cpu:     1
        Memory:  2Gi
      Requests:
        Cpu:     1
        Memory:  2Gi
  Service Type:  NodePort
Status:
  Cluster Service:       kdhs-7hkcl
  Generation UID:        bf08b83b-512a-42ef-9cad-9ff750e0a90b
  Last Connection Hash:  4

In [11]:
kubectl get all -l kubedirector.hpe.com/kdcluster=$clusterName

NAME               READY   STATUS    RESTARTS   AGE
pod/kdss-x7fx4-0   1/1     Running   0          104s

NAME                     TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)                       AGE
service/kdhs-7hkcl       ClusterIP   None            <none>        8888/TCP                      104s
service/s-kdss-x7fx4-0   NodePort    10.106.117.62   <none>        22:32013/TCP,8888:32674/TCP   104s

NAME                          READY   AGE
statefulset.apps/kdss-x7fx4   1/1     105s


Your instance of the KubeDirector Application virtual cluster is made up of a **StatefulSet**, a **POD** (a cluster node) and a **NodePort Service** per service role member (Controller), and a **headless service** for the application cluster.   

* The ClusterIP service is the headless service required by a Kubernetes StatefulSet to work. It maintains a stable POD network identity (i.e.: persistence of the hostname of the PODs across PODs rescheduling).
* The NodePort service exposes the Notebook application service with token-based authorization outside the Kubernetes cluster. 

HPE Ezmeral Container Platform automatically maps the NodePort Service endpoints to the HPE Ezmeral Container Platform gateway (haproxy) host.

#### Get the gateway mapped application service endpoint and the Authentication token to connect to your local Jupyter Notebook:
To get a report on all services related to a specific virtual KubeDirector cluster, you can use a form of **kubectl describe** that matches against a value of the **kubedirector.hpe.com/kdcluster=YourClusterApplicationName** label.

In [12]:
#
# Getting the service endpoint URL:
#
JupyterAppURL=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep gateway/8888 | awk '{print $2}')
JupyterAppPort=$(echo $JupyterAppURL | cut -d':' -f 2) # extract the gateway re-mapped port value.
myJupyterApp_endpoint="https://$gateway_host:$JupyterAppPort"
echo "Your application service endpoint re-mapped port is: "$JupyterAppPort
#echo "Your Intranet application service endpoint is: "$myJupyterApp_endpoint
echo "Your Jupyter Notebook service endpoint URL is: https://"$Internet_access:$JupyterAppPort
#
# Getting the auth-token:
#
JupyterAuthToken=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep kd-auth-token | awk '{print $2}')
echo "Your Jupyter Notebook service authentication token is: "$JupyterAuthToken

Your application service endpoint re-mapped port is: 10146
Your Jupyter Notebook service endpoint URL is: https://notebooks2.hpedev.io:10146
Your Jupyter Notebook service authentication token is: 32be7733e1ff90be0168b9fc29d64ecf


## Connect to your local Jupyter Notebook web UI and Upload code files
Click the service endpoint URL above to connect to your Jupyter Notebook server.
This opens a Jupyter Notebook login screen in a new browser tab. Login with the authentication token above (copy/paste the token value above) and follow instructions below to upload the code files (downloaded at the beginning of this lab part) to your local Jupyter Notebook server.

![Jupyter-Notebook-Login](Pictures/Jupyter-Notebook-Login.png)

Click on **Upload** button on the top right of your local Jupyter Notebook server.

![Jupyter-Notebook-Upload](Pictures/Jupyter-Notebook-Upload.png)

Select the Lab 3 python code file **3-WKSHP-K8s-ML-Pipeline-Model-Training.ipynb** from your laptop downloaded at the beginning of this lab, and click on **Upload** button as shown here:

![Lab3 code upload](Pictures/Jupyter-Notebook-Upload-file.png)

Repeat the same steps to upload the files **XGB_Scoring.py**, **XGB_Scoringv2.py** and **ML-Workflow.jpg** from your laptop downloaded at the beginning of this lab.

### <font color="red">Now, from your local Jupyter Notebook, open the notebook **3-WKSHP-K8s-ML-Pipeline-Model-Training.ipynb** and follow the instructions from the notebook to build, train and test the model.</font>

Once your model is trained and saved to a file, follow the instructions in Lab 4 to deploy you trained model:

* [Lab 4 Model Registry and Deployment](4-WKSHP-K8s-ML-Pipeline-Register-Model-Deployment.ipynb)

## Summary

In this lab, we have shown you how you, **as tenant user**, can deploy a local Jupyter Notebook virtual cluster and attach it to a shared distributed training cluster using _**Connections**_ stanza in KubeDirector Cluster manifest YAML file. This local Jupyter Notebook will be used in the next lab to do model training on the tenant-shared training cluster.